In [2]:
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler as ss
from sklearn.utils import shuffle

In [3]:
data=pd.read_csv("./StudentsPerformance.csv")
#data=data.drop(["lunch"],axis=1)
data=shuffle(data)
uniq_labels=data["gender"].unique()
labels=data.pop("gender")
labels=np.array(labels)
data=pd.get_dummies(data)
data=ss().fit_transform(data)


In [4]:
shalabels = np.zeros((len(labels), len(uniq_labels)))

In [5]:
for i in range(len(uniq_labels)):
    x = np.where(labels == uniq_labels[i])
    shalabels[x,i] = 1

shalabels[:15]

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]])

In [6]:
features = torch.tensor(np.array(data), dtype=torch.float32)
labels = torch.tensor(shalabels, dtype=torch.float32)

In [7]:
labels.shape

torch.Size([1000, 2])

In [8]:
model= torch.nn.Sequential(
    torch.nn.Linear(18,8),
    torch.nn.Softmax(),
    torch.nn.Linear(8,4),
    torch.nn.Softmax(),
    torch.nn.Linear(4,2),
    torch.nn.Softmax(dim=-1)
)
preds=model(features)
print(preds[:5])

tensor([[0.4591, 0.5409],
        [0.4572, 0.5428],
        [0.4595, 0.5405],
        [0.4567, 0.5433],
        [0.4583, 0.5417]], grad_fn=<SliceBackward0>)


c:\Users\v.jahnavi\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [9]:
loss_fn = torch.nn.BCELoss()

loss = loss_fn(preds, labels)
print(loss)

tensor(0.6933, grad_fn=<BinaryCrossEntropyBackward0>)


In [10]:
optimize= torch.optim.SGD(model.parameters(),lr=0.1e-1)

In [11]:
from torch import no_grad

epochs=100000
for epoch in range(epochs):
    preds=model(features)
    loss=loss_fn(preds,labels)
    loss.backward()
    optimize.step()
    optimize.zero_grad()
    with torch.no_grad():
        if (epoch+1)%(epochs//10)==0:
            right=0
            for i in range(len(preds)):
                for j in range(len(preds[i])): 
                    if preds[i][j]==max(preds[i]):
                        if(labels[i][j]==1):    
                            right+=1
            print("Loss",loss)
            print("Accuracy :",round(right*100/len(preds),5))

Loss tensor(0.6896, grad_fn=<BinaryCrossEntropyBackward0>)
Accuracy : 51.8
Loss tensor(0.5967, grad_fn=<BinaryCrossEntropyBackward0>)
Accuracy : 74.2
Loss tensor(0.2195, grad_fn=<BinaryCrossEntropyBackward0>)
Accuracy : 91.4


In [ ]:
torch.save(model.state_dict(),"genderchal.pth")